In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

In [2]:
random_state = 7
cv_method = StratifiedKFold(n_splits=5)

# Training of tsfresh-selected Training Max 1 years and eGFR 4 times

In [3]:
df = pd.read_csv('../data/train_all_1_4.gz')
df.shape

(5603, 49610)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5603 entries, 0 to 5602
Columns: 49610 entries, id to cat8
dtypes: float64(49588), int64(2), object(20)
memory usage: 2.1+ GB


In [5]:
df['aki_1'].fillna(value=0, inplace=True)
df['aki_2'].fillna(value=0, inplace=True)
df['aki_3'].fillna(value=0, inplace=True)

In [6]:
df['gn'].value_counts()

no                 5181
igan                172
anca                 76
lupus nephritis      39
membranous           33
unspecific           30
fsgs                 28
mcd                  21
mp/mcgn              16
fgn/itg               5
pign                  2
Name: gn, dtype: int64

In [7]:
df.loc[df.gn == 'pign', 'gn'] = 'unspecific'
df.loc[df.gn == 'fgn/itg', 'gn'] = 'unspecific'

In [8]:
percent_missing = df.isnull().sum()*100 / len(df)
missing_df = pd.DataFrame({'column_name': df.columns, 'percent_miss': percent_missing})

In [9]:
missing_df.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
"sta_dia__fft_coefficient__attr_""""""""abs""""""""__coeff_47","sta_dia__fft_coefficient__attr_""""""""abs""""""""__co...",100.0
"sta_hr__fft_coefficient__attr_""""""""real""""""""__coeff_17","sta_hr__fft_coefficient__attr_""""""""real""""""""__co...",100.0
"timedprotein__fft_coefficient__attr_""""""""abs""""""""__coeff_9","timedprotein__fft_coefficient__attr_""""""""abs""""""...",100.0
"timedprotein__fft_coefficient__attr_""""""""abs""""""""__coeff_8","timedprotein__fft_coefficient__attr_""""""""abs""""""...",100.0
"timedprotein__fft_coefficient__attr_""""""""abs""""""""__coeff_7","timedprotein__fft_coefficient__attr_""""""""abs""""""...",100.0
...,...,...
htn,htn,0.0
aki_3,aki_3,0.0
aki_2,aki_2,0.0
aki_1,aki_1,0.0


### Number of patients reaching ESKD within training period 

In [10]:
df['cat1'].value_counts()

non_eskd    4409
eskd        1194
Name: cat1, dtype: int64

In [11]:
df.set_index('id', inplace=True)

In [12]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        df[col] = df[col].astype('category')
    return df

def drop_missing(df):
    threshold = len(df)*0.6
    df.dropna(axis=1, thresh=threshold, inplace=True)
    return df

def copy_df(df):
    return df.copy()

In [13]:
df_cleaned = (df.pipe(copy_df).pipe(drop_missing).pipe(to_category))
df_cleaned.shape

(5603, 5990)

In [14]:
df_rm_intrain = df_cleaned.query("cat1 == 'non_eskd'")
df_rm_intrain.shape

(4409, 5990)

In [15]:
df_rm_intrain.to_csv('../data/cleaned60_all_1_4.gz', index=True, compression='gzip')

In [16]:
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4409 entries, 3916 to 23864
Columns: 5990 entries, mchc__variance_larger_than_standard_deviation to cat8
dtypes: category(20), float64(5969), int64(1)
memory usage: 200.9 MB


In [112]:
df_rm_intrain = (df_rm_intrain.pipe(copy_df).pipe(to_category))
df_rm_intrain.shape

(4409, 8985)

### Number of patients reaching ESKD in at 1y post-training 

In [17]:
df_rm_intrain['cat2'].value_counts()

non_eskd    4282
eskd         127
Name: cat2, dtype: int64

### Number of patients reaching ESKD in 2y post-training

In [18]:
df_rm_intrain['cat3'].value_counts()

non_eskd    4174
eskd         235
Name: cat3, dtype: int64

### Number of patients reaching ESKD in 5y post-training

In [19]:
df_rm_intrain['cat6'].value_counts()

non_eskd    3918
eskd         491
Name: cat6, dtype: int64

In [20]:
percent_missing = df_rm_intrain.isnull().sum()*100 / len(df_rm_intrain)
missing_df_rm_intrain = pd.DataFrame({'column_name': df_rm_intrain.columns, 'percent_miss': percent_missing})

In [21]:
missing_df_rm_intrain.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
caphos_product__autocorrelation__lag_0,caphos_product__autocorrelation__lag_0,47.153550
caphos_product__autocorrelation__lag_1,caphos_product__autocorrelation__lag_1,47.153550
caphos_product__fourier_entropy__bins_100,caphos_product__fourier_entropy__bins_100,47.130869
caphos_product__fourier_entropy__bins_10,caphos_product__fourier_entropy__bins_10,47.130869
caphos_product__fourier_entropy__bins_5,caphos_product__fourier_entropy__bins_5,47.130869
...,...,...
dkd,dkd,0.000000
htn,htn,0.000000
aki_3,aki_3,0.000000
aki_2,aki_2,0.000000


## Features and Target Input for Training 

In [22]:
dropped_cols = list(df_rm_intrain.columns[-18:])
dropped_cols.remove('age.init')
dropped_cols

['egfr.y',
 'cat0.5',
 'cat1',
 'cat1.5',
 'cat2',
 'cat2.5',
 'cat3',
 'cat3.5',
 'cat4',
 'cat4.5',
 'cat5',
 'cat5.5',
 'cat6',
 'cat6.5',
 'cat7',
 'cat7.5',
 'cat8']

In [23]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat6']
X.shape, y.shape

((4409, 5973), (4409,))

In [24]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3916     no
3918     no
3921     no
3924     no
3930     no
         ..
23760    no
23771    no
23780    no
23807    no
23864    no
Name: cat6, Length: 4409, dtype: category
Categories (2, object): ['yes', 'no']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3086, 5973), (1323, 5973), (3086,), (1323,))

In [26]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((3086,), (1323,))

In [27]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2742, 1: 344}

In [28]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

7.97093023255814

In [29]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['number']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 5969)

In [30]:
preprocess = ColumnTransformer([('numerical', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(3086, 5984)

In [31]:
X_test_processed = preprocess.transform(X_test)

In [32]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(sampling_strategy=0.5, random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [33]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling',
                 SMOTETomek(random_state=7, sampling_strategy=0.5)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=7.97093023255814, subsample=1,
                               tree_method='exact', use_label_encoder=False,
      

In [34]:
y_pred = pipeline.predict(X_test_processed)

In [35]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[1119,   57],
       [  91,   56]])

In [36]:
roc_auc_score(y_test, y_pred)

0.6662414965986394

In [37]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      1176
           1       0.50      0.38      0.43       147

    accuracy                           0.89      1323
   macro avg       0.71      0.67      0.68      1323
weighted avg       0.88      0.89      0.88      1323



In [38]:
print(matthews_corrcoef(y_test, y_pred))

0.37385212797150785


In [39]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((5984,), numpy.ndarray)

In [40]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

5984

In [41]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
3984,creatinine__quantile__q_0.9,0.115063
3979,creatinine__quantile__q_0.3,0.046660
0,mchc__variance_larger_than_standard_deviation,0.019250
4221,egfr__quantile__q_0.6,0.016217
323,mcv__cwt_coefficients__coeff_0__w_10__widths_(...,0.015140
4877,haematocrit__symmetry_looking__r_0.1,0.014002
3927,creatinine__maximum,0.013635
3929,creatinine__benford_correlation,0.012292
3187,"alkphos__linear_trend__attr_""""""""pvalue""""""""",0.011878
4251,egfr__cwt_coefficients__coeff_1__w_2__widths_(...,0.011755


In [42]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [43]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling', SMOTETomek(random_state=7)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=7.97093023255814, subsample=1,
                               tree_method='exact', use_label_encoder=False,
                               validate_parame

In [44]:
y_pred = pipeline.predict(X_test_processed)

In [45]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[1115,   61],
       [  82,   65]])

In [46]:
roc_auc_score(y_test, y_pred)

0.6951530612244898

In [47]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1176
           1       0.52      0.44      0.48       147

    accuracy                           0.89      1323
   macro avg       0.72      0.70      0.71      1323
weighted avg       0.89      0.89      0.89      1323

